In [1]:
# Installing the dependencies 
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00


In [2]:
# Downloading pretrained hugging face model for sentiment analysis and using it to make sample predictions 
import torch
from transformers import pipeline

# Check if a GPU is available
if torch.cuda.is_available():
    device = 0  # set the device to the first GPU
else:
    device = -1  # use the CPU if no GPU is available

# Set up the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", device=device)

# Define the input data
data = ["I love you", "I hate you"]

# Run the sentiment analysis pipeline on the input data
sentiment_pipeline(data)



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [3]:
%cd /content/drive/MyDrive
import pandas as pd 

/content/drive/MyDrive


In [4]:
# reading the data as pandas dataframe object
data = pd.read_csv("twcs.csv")

In [5]:
data.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

In [9]:
# Start labeling data. It may take while to run
from transformers import pipeline
from tqdm.auto import tqdm

def label_tweets(tweets):
    labels = []
    result = sentiment_pipeline(tweets)
    for res in result: 
        labels.append(res["label"])
    return labels

# Create a progress bar
pbar = tqdm(total=len(data))

# Split the tweets into batches
batch_size = 100
num_batches = len(data) // batch_size
if len(data) % batch_size != 0:
    num_batches += 1

labels = []
# Process the tweets in batches and update the progress bar
for i in range(num_batches):
    start = i * batch_size
    end = min((i+1) * batch_size, len(data))
    tweets = data.iloc[start:end]["text"].tolist()
    temp_labels = label_tweets(tweets)
    labels = labels+temp_labels
    # Update the progress bar for each batch
    pbar.update(len(tweets))

data["label"] = labels

# Close the progress bar
pbar.close()

  0%|          | 0/2811774 [00:00<?, ?it/s]

In [ ]:
# Saving the labeled dataset
data.to_csv("labeled_twcs.csv")